## Instalar librerías


In [1]:
# !python -m pip install Cython
# !python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
# !python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
# !python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
# !python -m pip install pyarrow
# !python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier

## Importar librerías


In [2]:
import numpy as np
import pandas as pd

#Constructor de curvas de luz
from lc_classifier.utils import LightcurveBuilder
#Compositor de Extractores
from lc_classifier.features import FeatureExtractorComposer
#Extractores Generales:
from lc_classifier.features import MHPSExtractor
from lc_classifier.features import PeriodExtractor
from lc_classifier.features import GPDRWExtractor
from lc_classifier.features import FoldedKimExtractor
from lc_classifier.features import HarmonicsExtractor
from lc_classifier.features import IQRExtractor
from lc_classifier.features import PowerRateExtractor
from lc_classifier.features import TurboFatsFeatureExtractor
#Extractores Supernova:
from lc_classifier.features import SupernovaeDetectionFeatureExtractor
from lc_classifier.features import SupernovaeDetectionAndNonDetectionFeatureExtractor
from lc_classifier.features import SNParametricModelExtractor
from lc_classifier.features import ZTFColorFeatureExtractor

from lc_classifier.features import GalacticCoordinatesExtractor
from lc_classifier.features import RealBogusExtractor



## Importar dataset

In [3]:
df_novas = pd.read_pickle('df_novas.pkl')

In [4]:
df_novas.head()

,oid,fid,magpsf,sigmapsf,magpsf_corr,sigmapsf_corr,mjd,isdiffpos,rb,ra,dec,sgscore1,classALeRCE
0,ZTF18aahvndq,1,15.385100,0.028791,15.385100,0.028791,58991.302095,1.0,0.770000,198.722678,27.008621,0.175976,SNIa
1,ZTF18aahvndq,1,15.712121,0.031231,15.712121,0.031231,59000.260914,1.0,0.894286,198.722679,27.008635,0.175976,SNIa
2,ZTF18aahvndq,1,16.072975,0.032161,16.072975,0.032161,59004.192674,1.0,0.907143,198.722653,27.008647,0.175976,SNIa
3,ZTF18aahvndq,1,17.708502,0.049931,17.708502,0.049931,59021.227604,1.0,0.947143,198.722660,27.008624,0.175976,SNIa
4,ZTF18aahvndq,1,15.617474,0.028520,15.617474,0.028520,58986.223646,1.0,0.837143,198.722660,27.008636,0.175976,SNIa


## Feature Extractor

In [5]:
bands = ['g', 'r']

feature_extractor = FeatureExtractorComposer(
    [   
        SupernovaeDetectionFeatureExtractor(bands),
        MHPSExtractor(bands), #Esta
        PeriodExtractor(bands, 0.01, 0.99, False, None, 20), #ESTA
        GPDRWExtractor(bands), #ESTA
        FoldedKimExtractor(bands), #ESTA
        HarmonicsExtractor(bands), #ESTA
        IQRExtractor(bands), #Esta
        PowerRateExtractor(bands), #ESTA
        TurboFatsFeatureExtractor(bands), #Esta
        SNParametricModelExtractor(bands), #  ESTA
        # ZTFColorFeatureExtractor(), #Esta
        # GalacticCoordinatesExtractor(), #Esta
        # RealBogusExtractor(), #Esta
        # SupernovaeDetectionAndNonDetectionFeatureExtractor(bands), #ESTA
    ]
)


## Definir Dataset Curvas de Luz

In [6]:
df_lightcurves = pd.DataFrame()
df_lightcurves['oid'] = df_novas["oid"]
df_lightcurves['band'] = df_novas['fid'].replace({1 : 'g', 2 : 'r'})
df_lightcurves['time'] = df_novas['mjd']
df_lightcurves['magnitude'] = df_novas['magpsf_corr']
df_lightcurves['error'] = df_novas['sigmapsf_corr']
df_lightcurves['isdiffpos'] = df_novas['isdiffpos']

df_lightcurves['magpsf'] = df_novas['magpsf']
df_lightcurves['sigmapsf'] = df_novas['sigmapsf']
df_lightcurves['rb'] = df_novas['rb']
df_lightcurves['ra'] = df_novas['ra']
df_lightcurves['dec'] = df_novas['dec']


# 'rb', 'error', 'magpsf', 'band', 'sigmapsf', 'time', 'ra', 'magnitude', 'dec', 'isdiffpos'

df_lightcurves = df_lightcurves.set_index('oid')

df_lightcurves.head()

,band,time,magnitude,error,isdiffpos,magpsf,sigmapsf,rb,ra,dec
oid,,,,,,,,,,
ZTF18aahvndq,g,58991.302095,15.385100,0.028791,1.0,15.385100,0.028791,0.770000,198.722678,27.008621
ZTF18aahvndq,g,59000.260914,15.712121,0.031231,1.0,15.712121,0.031231,0.894286,198.722679,27.008635
ZTF18aahvndq,g,59004.192674,16.072975,0.032161,1.0,16.072975,0.032161,0.907143,198.722653,27.008647
ZTF18aahvndq,g,59021.227604,17.708502,0.049931,1.0,17.708502,0.049931,0.947143,198.722660,27.008624
ZTF18aahvndq,g,58986.223646,15.617474,0.028520,1.0,15.617474,0.028520,0.837143,198.722660,27.008636


## Extraer features

In [7]:
features = feature_extractor.compute_features(df_lightcurves)
features

/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: overflow encountered in square
  return np.sum(alpha ** 2 / self._d)
/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(alpha ** 2 / self._d)
/home/jovyan/work/mds/Proyecto/src/lc-classifier/lc_classifier/features/extractors/gp_drw_extractor.py:52: RuntimeWarning: overflow encountered in exp
  theta = np.exp(params)
/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(alpha ** 2 / self._d)
/opt/conda/lib/python3.9/site-packages/celerite2/core.py:427: RuntimeWarning: invalid value encountered in double_scalars
  return self._norm - 0.5 * self._do_norm(y - self._mean_value)
/opt/conda/lib/python3.9/site-packages/celerite2/numpy.py:108: RuntimeWarning: overflow encountered in square
  return np.sum(alpha ** 2 / self._d)
/home/jovyan/work/mds/Pr

,delta_mag_fid_g,delta_mjd_fid_g,first_mag_g,mean_mag_g,min_mag_g,n_det_g,n_neg_g,n_pos_g,positive_fraction_g,delta_mag_fid_r,...,SPM_tau_rise_g,SPM_tau_fall_g,SPM_chi_g,SPM_A_r,SPM_t0_r,SPM_gamma_r,SPM_beta_r,SPM_tau_rise_r,SPM_tau_fall_r,SPM_chi_r
oid,,,,,,,,,,,,,,,,,,,,,
ZTF17aadlxmv,0.489893,42.055613,17.454949,17.295473,16.994532,18.0,0.0,18.0,1.0,1.223410,...,2.329819,11.688814,0.007553,0.311686,6.958042,21.231877,0.416775,3.677212,30.091595,0.594409
ZTF18aaaqexr,0.121316,18.870671,16.733905,16.711534,16.633707,9.0,0.0,9.0,1.0,0.015051,...,1.205330,17.585408,0.281999,0.292700,-8.394263,9.801841,0.447691,8.599627,32.415046,NaN
ZTF18aacdbzx,0.123097,19.991725,18.178415,18.159091,18.096627,14.0,0.0,14.0,1.0,0.181092,...,3.344789,24.821387,0.033287,0.114159,-5.114745,13.845660,0.063725,10.514426,20.820954,0.044529
ZTF18aadmssd,1.015765,59.951100,16.399704,17.049047,16.399704,16.0,0.0,16.0,1.0,0.597693,...,2.939171,22.204705,0.010855,2.067952,-22.606538,39.255646,0.755130,17.087418,46.760688,1.111489
ZTF18aadzfso,0.364548,32.999224,17.839005,17.710422,17.475610,24.0,0.0,24.0,1.0,0.140507,...,3.554762,14.225595,0.028317,0.257985,4.309857,18.030787,0.606824,3.959666,85.046761,0.027387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20abgbxfm,2.981503,68.950197,18.893476,19.555296,18.296597,58.0,0.0,58.0,1.0,2.125096,...,3.176700,24.208459,0.077253,0.287074,4.316306,14.243016,0.570416,4.845102,28.891702,0.040567
ZTF20abgdtmv,2.669654,33.020729,18.728144,20.046770,18.663546,26.0,0.0,26.0,1.0,1.719614,...,6.300765,10.058112,0.013802,0.245716,12.104638,2.379998,0.340974,6.101358,14.828762,0.039753
ZTF20abgfekk,1.314085,31.928044,18.671485,17.593371,17.357400,32.0,0.0,32.0,1.0,1.096404,...,3.415255,19.974875,0.045213,0.401462,5.696663,35.855799,0.521981,3.882842,30.064585,0.056440


## Juntar features extraidas con columnas de sgscore1 y clase

In [8]:
features = features.reset_index()
df_final_features = pd.merge(features, df_novas[['oid', 'sgscore1', 'classALeRCE']], 
                     left_on='oid', right_on='oid')

In [9]:
df_final_features

,oid,delta_mag_fid_g,delta_mjd_fid_g,first_mag_g,mean_mag_g,min_mag_g,n_det_g,n_neg_g,n_pos_g,positive_fraction_g,...,SPM_chi_g,SPM_A_r,SPM_t0_r,SPM_gamma_r,SPM_beta_r,SPM_tau_rise_r,SPM_tau_fall_r,SPM_chi_r,sgscore1,classALeRCE
0,ZTF17aadlxmv,0.489893,42.055613,17.454949,17.295473,16.994532,18.0,0.0,18.0,1.0,...,0.007553,0.311686,6.958042,21.231877,0.416775,3.677212,30.091595,0.594409,0.174589,SNIa
1,ZTF17aadlxmv,0.489893,42.055613,17.454949,17.295473,16.994532,18.0,0.0,18.0,1.0,...,0.007553,0.311686,6.958042,21.231877,0.416775,3.677212,30.091595,0.594409,0.174589,SNIa
2,ZTF17aadlxmv,0.489893,42.055613,17.454949,17.295473,16.994532,18.0,0.0,18.0,1.0,...,0.007553,0.311686,6.958042,21.231877,0.416775,3.677212,30.091595,0.594409,0.174589,SNIa
3,ZTF17aadlxmv,0.489893,42.055613,17.454949,17.295473,16.994532,18.0,0.0,18.0,1.0,...,0.007553,0.311686,6.958042,21.231877,0.416775,3.677212,30.091595,0.594409,0.174589,SNIa
4,ZTF17aadlxmv,0.489893,42.055613,17.454949,17.295473,16.994532,18.0,0.0,18.0,1.0,...,0.007553,0.311686,6.958042,21.231877,0.416775,3.677212,30.091595,0.594409,0.174589,SNIa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128713,ZTF20abhvnzc,1.237353,14.989954,18.093634,18.536102,17.967752,18.0,0.0,18.0,1.0,...,0.103456,0.369196,-4.863622,15.287332,0.004390,12.511147,6.616601,0.650528,0.328119,SNIa
128714,ZTF20abhvnzc,1.237353,14.989954,18.093634,18.536102,17.967752,18.0,0.0,18.0,1.0,...,0.103456,0.369196,-4.863622,15.287332,0.004390,12.511147,6.616601,0.650528,0.328119,SNIa
128715,ZTF20abhvnzc,1.237353,14.989954,18.093634,18.536102,17.967752,18.0,0.0,18.0,1.0,...,0.103456,0.369196,-4.863622,15.287332,0.004390,12.511147,6.616601,0.650528,0.328119,SNIa
128716,ZTF20abhvnzc,1.237353,14.989954,18.093634,18.536102,17.967752,18.0,0.0,18.0,1.0,...,0.103456,0.369196,-4.863622,15.287332,0.004390,12.511147,6.616601,0.650528,0.328119,SNIa


## Exportar dataset

In [10]:
df_final_features.to_pickle('features_extraidas.pkl')